In [14]:
import polars as pl
import pandas as pd
from dotenv import load_dotenv
from api.internal_api import SynthetixAPI, get_db_config

load_dotenv()

# constants
DATA_LOC = "~/repos/data/parquet-data/indexers"
db_config = get_db_config(streamlit=False)
api = SynthetixAPI(db_config, streamlit=False)


In [15]:
# get tlx data
df_pl = pl.scan_parquet(
    [
        f"{DATA_LOC}/raw/optimism_mainnet/tlx/**/*_event_minted.parquet",
        f"{DATA_LOC}/raw/optimism_mainnet/tlx/**/*_event_redeemed.parquet",
    ],
    include_file_paths="file_location",
)
df = df_pl.collect()
df_tlx = df.to_pandas()


In [16]:
df_tlx.columns

Index(['id', 'blockNumber', 'blockTimestamp', 'transactionHash', 'contract',
       'eventName', 'account', 'leveragedTokenAmount', 'baseAssetAmount',
       'file_location'],
      dtype='object')

In [3]:
df_tlx["ts"] = df_tlx["blockTimestamp"]
df_tlx["market"] = (
    df_tlx["file_location"]
    .apply(lambda x: x.split("/")[-1].split("_")[0][:-1])
    .str.upper()
)


In [4]:
# get prices
df_prices = api._run_query(
    """
SELECT distinct ts,
    block_number,
    market,
    last(price) over(partition by market order by ts) as price
FROM prod_optimism_mainnet.fct_v2_trades_optimism_mainnet
    """
)
print(df_prices)


                               ts  block_number market     price
0       2023-07-17 17:20:57+00:00     107007840    ARB    1.2687
1       2023-04-18 06:33:03+00:00      91791929    ARB    1.7780
2       2023-05-14 06:34:21+00:00      98122805    ARB    1.1728
3       2023-08-08 12:19:07+00:00     107949185    APE    1.8259
4       2024-02-13 23:07:03+00:00     116133423    APE    1.5134
...                           ...           ...    ...       ...
1039568 2024-07-03 22:05:49+00:00     122222786    ETH 3296.5176
1039569 2023-12-20 19:17:09+00:00     113750526   AVAX   43.9023
1039570 2023-09-06 17:00:33+00:00     109210428    ETH 1622.2090
1039571 2024-03-16 02:48:41+00:00     117479272    SEI    1.1326
1039572 2024-11-14 05:40:17+00:00     127982020    SUI    3.4965

[1039573 rows x 4 columns]


In [5]:
df_prices.head()

,ts,block_number,market,price
0,2023-07-17 17:20:57+00:00,107007840,ARB,1.2687
1,2023-04-18 06:33:03+00:00,91791929,ARB,1.7780
2,2023-05-14 06:34:21+00:00,98122805,ARB,1.1728
3,2023-08-08 12:19:07+00:00,107949185,APE,1.8259
4,2024-02-13 23:07:03+00:00,116133423,APE,1.5134


In [6]:
df_tlx.head()

,id,blockNumber,blockTimestamp,transactionHash,contract,eventName,account,leveragedTokenAmount,baseAssetAmount,file_location,ts,market
0,0120085092-000129-c067a,120085092,2024-05-15 10:29:21+00:00,0x8f6db5f7afa823d56161664162db3a924cd029accf2b...,0x169d4884be225b322963912df3641948143ff92b,Minted,0x559822cf7213bC2DDa0aeCffA0b66Bd083C169CD,100000000000000000000,99998796635982988401,/Users/Troy/repos/data/parquet-data/indexers/r...,2024-05-15 10:29:21+00:00,BTC
1,0120085440-000169-944fa,120085440,2024-05-15 10:40:57+00:00,0xfd3af02755c493352bd3c11092606d25e1b807cb4b13...,0x169d4884be225b322963912df3641948143ff92b,Minted,0x559822cf7213bC2DDa0aeCffA0b66Bd083C169CD,50000000000000000000,50538751259711503056,/Users/Troy/repos/data/parquet-data/indexers/r...,2024-05-15 10:40:57+00:00,BTC
2,0120227737-000021-898ea,120227737,2024-05-18 17:44:11+00:00,0x6f06b2eaf5fd925895d68c112a2fd7485eaadcb08622...,0x169d4884be225b322963912df3641948143ff92b,Minted,0x07Aeeb7E544A070a2553e142828fb30c214a1F86,10000000000000000000,9555743335319838006,/Users/Troy/repos/data/parquet-data/indexers/r...,2024-05-18 17:44:11+00:00,BTC
3,0120230183-000106-78d76,120230183,2024-05-18 19:05:43+00:00,0x4f13e17fca0d522bcd6ad02b200f5bea026d6459b446...,0x169d4884be225b322963912df3641948143ff92b,Minted,0x9e5A57470A2a136a3d4De39e1996E05944C4b1fb,98947628004605507454,94572586569905948466,/Users/Troy/repos/data/parquet-data/indexers/r...,2024-05-18 19:05:43+00:00,BTC
4,0120230411-000029-db77c,120230411,2024-05-18 19:13:19+00:00,0x5b8ba20572d01174e6312a6de4dcbbde21d973119dd1...,0x169d4884be225b322963912df3641948143ff92b,Minted,0x9e5A57470A2a136a3d4De39e1996E05944C4b1fb,2832000000000000000000,2717933929106361752497,/Users/Troy/repos/data/parquet-data/indexers/r...,2024-05-18 19:13:19+00:00,BTC


In [7]:
# Convert 'ts' columns to integer timestamps in milliseconds
df_tlx["ts_int"] = df_tlx["ts"].astype("int64")
df_prices["ts_int"] = df_prices["ts"].astype("int64") // 10**6

# Sort DataFrames by 'ts_int' before merging
df_tlx_sorted = df_tlx.sort_values("ts_int")
df_prices_sorted = df_prices.sort_values("ts_int")

# Perform asof merge on 'ts_int'
df_merged = pd.merge_asof(
    df_tlx_sorted, df_prices_sorted, on="ts_int", by="market", direction="backward"
)

# Convert 'ts_int' back to datetime
df_merged["ts"] = pd.to_datetime(df_merged["ts_int"], unit="ms", utc=True)

# Drop the 'ts_int' column if no longer needed
df_merged = df_merged.drop(columns=["ts_int"])


In [8]:
df_merged["volume"] = df_merged["leveragedTokenAmount"].apply(lambda x: int(x) / 1e18)

In [9]:
df_merged.groupby("market")["volume"].sum().sort_values(ascending=False)

market
SOL       73640167.9628
ETH       38028447.3733
BTC       27374002.9477
PEPE       9490307.9370
DOGE       3857734.7493
SUI        2081530.5400
OP          770563.5337
LINK        497864.7436
ETHBTC1     255571.1194
SEI         235991.9719
ETHBTC      105470.1066
RUNE         76205.7347
Name: volume, dtype: float64

In [10]:
# top accounts all time
top_accounts = (
    df_merged.groupby("account")["volume"].sum().sort_values(ascending=False).head(25)
)
top_accounts_30 = (
    df_merged[df_merged["ts"] > pd.Timestamp.now(tz="UTC") - pd.Timedelta(days=30)]
    .groupby("account")["volume"]
    .sum()
    .sort_values(ascending=False)
    .head(25)
)

In [11]:
print(top_accounts)

account
0x19A610D0Bf96530003c1521771223ED5488e7f80   13225686.6384
0x081CDbc7b1d8972704F27b2840df30808c73Eb8F   12910448.5951
0x28b82692c8010C0f73b32822Ed1e480c9032b6CB    8733373.1092
0x25C68aC0Cd0E2ACC69f1E16372c0876FD2ddcC02    8278827.4631
0xb3E1A2299fFe8bB914E426023B877c053116e407    7948689.3126
0x76ec668AF27f6426eF3478DceC55f8255baFD6E2    6157297.7583
0xcbA16808cb1F7a21fD944005F75d6a171cF2e51d    4740573.0175
0x6262210738C6ceDd9813aFAd8D4684f8879266F2    4424642.0433
0x829d069b91e0986DbA484C344187F2021132c65a    2503682.3620
0x43B2ff7d7AB907AC88FbFC1BE9422c817732561B    2178245.4676
0xDF83756D2c07837a7a3366738520E53128Cefc45    1603954.2379
0x6bCEc4D5Ec1858C5b32Fe46E1c32DCE6BE23A528    1411657.1129
0x20303881946dE067A36551C2e7476215BAd4fca6    1317815.0010
0xEf7f5f79550b701e84988fcbE474dC5614E03fe1    1269175.7007
0xFeD00C6a94B5807B9a871cd2c02c12be1D92887a    1230528.4981
0x4b485AB85070123fcf714BDBB21E31f1ebE1FE20    1210446.8513
0xF0BD8676AF8cFb5dF1720fa6a783900379C08184    11

In [12]:
print(top_accounts_30)

account
0x28b82692c8010C0f73b32822Ed1e480c9032b6CB   4248950.7373
0x6262210738C6ceDd9813aFAd8D4684f8879266F2   4110944.5084
0x829d069b91e0986DbA484C344187F2021132c65a   1887369.0469
0x4b485AB85070123fcf714BDBB21E31f1ebE1FE20   1210446.8513
0x5aF5Df60d112089a8194123e5CEE547c383c038c    926929.1471
0x2020f3c2D31DC374709b830Cd1D67EA1D5a3997C    672935.3681
0x1140321cCE279B4a2158571eb377669Def562Ac4    647641.5611
0x43B2ff7d7AB907AC88FbFC1BE9422c817732561B    631023.5949
0x0364b6466A81496B651cbc585b1c7699B5876051    419707.0355
0x68FB040f644a531fe8232FCd7D848254e7b4D021    395520.2608
0xEFa9817089e6Dd0404386d611CE80426BEE2588E    339587.6212
0xEc8Bc2a6ae1460e385F68b3e0594B33252372151    301963.1889
0xb12200e7d1ec6029c56685B956E6943E4A86B7a3    283150.6719
0xBddcD6Cf8107C969a141DFf1776D1c697Aea9fF4    259715.9176
0x41549A400e3f28E5F153F07361930e684daF6652    234222.9939
0x24056c7bEe4BC7129d4271cA9186130edc81AFdD    228549.5010
0xC04d6DB6CFb0e7E0366fC7Ec9945A51967fE14E5    223260.8822
0x8FB0

In [13]:
top_accounts.to_csv("top_accounts.csv")
top_accounts_30.to_csv("top_accounts_30.csv")